In [34]:
%pip install -vvv langchain langchain-openai langchain_chroma langchain_community langchainhub beautifulsoup4 rich

Using pip 25.0.1 from c:\Users\Jauni\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip (python 3.13)
Non-user install because site-packages writeable
Created temporary directory: C:\Users\Jauni\AppData\Local\Temp\pip-build-tracker-weowavvp
Initialized build tracking at C:\Users\Jauni\AppData\Local\Temp\pip-build-tracker-weowavvp
Created build tracker: C:\Users\Jauni\AppData\Local\Temp\pip-build-tracker-weowavvp
Entered build tracker: C:\Users\Jauni\AppData\Local\Temp\pip-build-tracker-weowavvp
Created temporary directory: C:\Users\Jauni\AppData\Local\Temp\pip-install-btba5a04
Created temporary directory: C:\Users\Jauni\AppData\Local\Temp\pip-ephem-wheel-cache-b8h78jqh
Created temporary directory: C:\Users\Jauni\AppData\Local\Temp\pip-unpack-oye9e4ye
Remote version of pip: 25.1.1
Local version of pip:  25.0.1
Was pip installed by pip? True
Removed build tracker: 'C:\\Users\\Jauni\\AppData\\Local\\Temp\\pip-build-tracker-weowavvp'
Note: you may need to restart the kernel to u


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables

token = os.getenv("SECRET")  # Replace with your actual token
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, base_url=endpoint, api_key=token)

In [17]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich import print

In [33]:
# Load, chunk and index the contents of the blog.
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2017-06-21-overview/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
splits = text_splitter.split_documents(docs)

# print(splits)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(
    model="text-embedding-3-small",
    base_url="https://models.inference.ai.azure.com",
    api_key=token,
))

In [26]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

print(prompt)

c:\Users\Jauni\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(
    input_variables=['context', 'question'],
    input_types={},
    partial_variables={},
    metadata={
        'lc_hub_owner': 'rlm',
        'lc_hub_repo': 'rag-prompt',
        'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'
    },
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'],
                input_types={},
                partial_variables={},
                template="You are an assistant for question-answering tasks. Use the following pieces of retrieved 
context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences 
maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
            ),
            additional_kwargs={}
        )
    ]
)

In [29]:
def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [30]:
rag_chain.invoke("What is Convolutional Neural Networks?")

[
    Document(
        id='78a9ae2a-83c8-4f31-b223-b678b09554ed',
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='Convolutional Neural Network#\nConvolutional neural networks, short for “CNN”, is a type of 
feed-forward artificial neural networks, in which the connectivity pattern between its neurons is inspired by the 
organization of the visual cortex system. The primary visual cortex (V1) does edge detection out of the raw visual 
input from the retina. The secondary visual cortex (V2), also called prestriate cortex, receives the edge features 
from V1 and extracts simple visual properties such as orientation, spatial frequency, and color. The visual area V4
handles more complicated object attributes. All the processed visual features flow into the final logic unit, 
inferior temporal gyrus (IT), for object recognition. The shortcut between V1 and V4 inspires a special type of CNN
with connections between non-adjacent layers: Residual Net (He, et al. 2016) containing “Residual Block” which 
supports some input of one layer to be passed to the component two layers later.'
    ),
    Document(
        id='690c45c7-4c0c-4411-98cf-f14179852526',
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='Convolutional neural networks, short for “CNN”, is a type of feed-forward artificial neural 
networks, in which the connectivity pattern between its neurons is inspired by the organization of the visual 
cortex system. The primary visual cortex (V1) does edge detection out of the raw visual input from the retina. The 
secondary visual cortex (V2), also called prestriate cortex, receives the edge features from V1 and extracts simple
visual properties such as orientation, spatial frequency, and color. The visual area V4 handles more complicated 
object attributes. All the processed visual features flow into the final logic unit, inferior temporal gyrus (IT), 
for object recognition. The shortcut between V1 and V4 inspires a special type of CNN with connections between 
non-adjacent layers: Residual Net (He, et al. 2016) containing “Residual Block” which supports some input of one 
layer to be passed to the component two layers later.'
    ),
    Document(
        id='97a36dad-fe56-48ac-a74c-88f0bdf6b079',
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='Illustration of the human visual cortex system. (Image source: Wang & Raj 
2017)\n\nConvolution is a mathematical term, here referring to an operation between two matrices. The convolutional
layer has a fixed small matrix defined, also called kernel or filter. As the kernel is sliding, or convolving, 
across the matrix representation of the input image, it is computing the element-wise multiplication of the values 
in the kernel matrix and the original image values. Specially designed kernels can process images for common 
purposes like blurring, sharpening, edge detection and many others, fast and efficiently.\n\n\nThe LeNet 
architecture consists of two sets of convolutional, activation, and pooling layers, followed by a fully-connected 
layer, activation, another fully-connected layer, and finally a softmax classifier (Image source: 
http://deeplearning.net/tutorial/lenet.html)'
    ),
    Document(
        id='f474f903-7159-4291-b40d-e5ebb890e920',
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='Illustration of the human visual cortex system. (Image source: Wang & Raj 
2017)\n\nConvolution is a mathematical term, here referring to an operation between two matrices. The convolutional
layer has a fixed small matrix defined, also called kernel or filter. As the kernel is sliding, or convolving, 
across the matrix representation of the input image, it is computing the element-wise multiplication of the values 
in the kernel matrix and the original i

'Convolutional Neural Networks (CNNs) are a type of artificial neural network inspired by the visual cortex, designed to process visual information efficiently. They use convolutional layers with small kernels or filters that slide over input images to detect features like edges and textures. CNNs are widely used in image recognition and related tasks due to their ability to automatically learn hierarchical visual features.'